<img src="./img/logo2.png"><br>
* ref : https://www.kaggle.com/rounakbanik/the-movies-dataset

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('./datasets/datasets_movie/movies_metadata.csv', low_memory=False)
data.head(2) # title, overview 

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
df = data[:20000].copy()

In [4]:
df['overview'].isnull().sum()

135

In [5]:
df['overview'] = df['overview'].fillna('')

### overview :  tf-idf 
* tf   : 문서1개기준   몇 번 단어가 있어
* idf  : 문저전체기준  나왔어?안나왔어

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])
print(tfidf_matrix.shape)
print(tfidf_matrix.toarray()[:1])

(20000, 47487)
[[0. 0. 0. ... 0. 0. 0.]]


In [7]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:2]

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ]])

In [8]:
ss = pd.Series(data.index, index=data['title']).drop_duplicates()
print(ss.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [9]:
idx = ss['Jumanji']
print(idx)


1


### 선택한 영화에 대해서 코사인 유사도를 이용하여, 가장 overview가 유사한 10개의 영화 찾기

In [10]:
def my_recommend(title, cosine_sim=cosine_sim):
    idx = ss[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))                       #title vs 모든영화 : 유사도계산
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)   # 유사도 정렬
    sim_scores = sim_scores[1:11]                                       # 유사도 top10      
    print(sim_scores)   
    movie_idx = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_idx]                            # 가장 유사한 영화 top10

In [11]:
my_recommend('Jumanji')

1
[(6166, 0.18004489861497824), (8801, 0.17893548282391922), (17223, 0.17626361556882922), (9503, 0.17184603122045175), (13601, 0.15998391697583217), (16843, 0.15976609460148378), (8079, 0.1586328230927603), (6055, 0.15508984041647622), (19726, 0.15310151784992126), (2486, 0.14983947719398977)]


6166                       Brainscan
8801                         Quintet
17223                 The Dark Angel
9503                       Word Wars
13601    The Mindscape of Alan Moore
16843                         DeVour
8079                         Masques
6055                Poolhall Junkies
19726                 Wreck-It Ralph
2486                        eXistenZ
Name: title, dtype: object